### Сторонние библиотеки использовать нельзя

### Задача 0 [Библиотека] (0.15 балла)  

**Условие:** 


В библиотеке хранятся книги и журналы. У каждой сущности есть общие характеристики, такие как: название, автор, жанр, число страниц, формат страниц, индекс редкости (от 1 до 10) и текст. Также у разных сущностей могут быть свои атрибуты. Хочется все редкие издания (индекс 9 или 10) дополнительно сохранять в некое хранилище (пусть json-файл), а также хочется понимать какую площадь занимает издание, если разложить все его страницы на полу.     


**Комментарий:**

Это задача с семинара на организацию иерархии классов. Идея в том, что нужно разделять сущности в зависимости от их применения. Например, есть книга как некий абстрактный объект, а есть библиотечная книга, у которой есть свои особенности. Также для сохранения книг в json нужно использвать классы-примеси.


Иерархия классов:

In [379]:

class ReadableEntity:
    
    PAGES_FORMAT = {
    'A1': (2048, 1024),
    'A2': (1024, 512),
    'A3': (512, 256),
    'A4': (297, 210),
    }
    
    def __init__(self, name, author, genre, n_pages, page_format, rare_index, text):
        
        self.name = name
        self.author = author
        self.genre = genre
        self.n_pages = n_pages
        self.page_format = page_format
        self.rare_index = rare_index
        self.text = text
    
    @property
    def square_area_mm(self):
        page_dims = ReadableEntity.PAGES_FORMAT[self.page_format]
        return self.n_pages * page_dims[0] * page_dims[1]
    
    @property
    def page_format(self):
        return self._page_format
    
    @page_format.setter
    def page_format(self, value):
        if ReadableEntity.PAGES_FORMAT.get(value) is None:
            raise ValueError('Page format not recognizeed')
        else:
            self._page_format = value
    
    @property
    def n_pages(self):
        return self._n_pages
    
    @n_pages.setter
    def n_pages(self, value):
        if isinstance(value, int):
            self._n_pages = value
        else:
            raise ValueError('N pages should be integer')
    
    @property
    def rare_index(self):
        return self._rare_index
    
    @rare_index.setter
    def rare_index(self, value):
        if isinstance(value, int):
            if 1 <= value <= 10:
                self._rare_index = value
            else:
                raise ValueError('Rare index should be in the range from 1 to 10')
        else:
            raise ValueError('Rare Index should be integer')
        
        
class Journal(ReadableEntity):
    
    def __init__(self, name, author, genre, n_pages, page_format, rare_index, text):
        super().__init__(name, author, genre, n_pages, page_format, rare_index, text)
        # here we could've defined other Journal properties as well

class Book(ReadableEntity):
    
    def __init__(self, name, author, genre, n_pages, page_format, rare_index, text):
        super().__init__(name, author, genre, n_pages, page_format, rare_index, text)
        # here we could've defined other Book properties as well

class Exporter:
    
    def write_or_print(self, method):
        method("{")
        for key in self.__dict__:
            method('\"{}\": \"{}\",'.format(key.lstrip('_'), self.__dict__[key]))
        method("}")
    def export_to_txt(self, file_path, is_write_to_sreen = False):
        if is_write_to_sreen == True:
            self.write_or_print(print)
        else:
            with open(file_path, 'a') as f:
                self.write_or_print(f.write)

    
class LibraryJournal(Journal, Exporter):
    
    RARE_ENTITIES = (9, 10)
    FILE_PATH = './data/'
    IS_WRITE_TO_SCREEN = True
    
    def __init__(self, name, author, genre, n_pages, page_format, rare_index, text):
    
        super().__init__(name, author, genre, n_pages, page_format, rare_index, text)
        if self.rare_index in self.RARE_ENTITIES:
            self.export_to_txt(self.FILE_PATH, self.IS_WRITE_TO_SCREEN)
    

class LibraryBook(Book, Exporter):
    
    RARE_ENTITIES = (9, 10)
    FILE_PATH = './data'
    IS_WRITE_TO_SCREEN = True
    
    def __init__(self, name, author, genre, n_pages, page_format, rare_index, text):
    
        super().__init__(name, author, genre, n_pages, page_format, rare_index, text)
        if self.rare_index in self.RARE_ENTITIES:
            self.export_to_txt(self.FILE_PATH, self.IS_WRITE_TO_SCREEN)

In [384]:
book = LibraryBook('harry potter', 'rowling', 'fiction', 1111, 'A4', 9, 'Once upon a time...')

{
"name": "harry potter",
"author": "rowling",
"genre": "fiction",
"n_pages": "1111",
"page_format": "A4",
"rare_index": "9",
"text": "Once upon a time...",
}


In [388]:
book = LibraryBook('harry potter', 'rowling', 'fiction', 1111, 'A4', 5, 'Once upon a time...')

In [389]:
book.square_area_mm

69293070

### Задача 1 [Размер объектов] (0 - 0.15 балла)  

**Условие:** 

Написать функцию получения реального объема занимаемой объектом памяти объектом. 


1) Для int, str, list, tuple, dict **(0.05 балла)**

2) Для всех типов **(+0.1 балла)**


**Комментарий:**

На занятиях не раз говорилось, что `sys.getsizeof` умеет находить размер простых объектов, но если речь идет об объектах, вроде list, то функция вернет не совсем то, что может ожидать разработчик, потому что список хранит указатели на объекты. 

*Пример:*
```
sys.getsizeof([]) == 64
sys.getsizeof(['aaaaaaa']) == 72
```
Но
```
sys.getsizeof('aaaaaaa') == 56
```


In [235]:
# def old_get_real_size_of(obj):
    
#     obj_ids = set()
#     size = sys.getsizeof(obj)
    
#     if type(obj) in (str, float, int):
#         return size
    
#     if type(obj) in (list, tuple):
        
#         for item in obj: 
#             item_id = id(item)
#             if item_id not in obj_ids:
#                 obj_ids.add(item_id)
#                 size += sys.getsizeof(item)
#         return size
    
#     if type(obj) == (dict):
        
#         for k, v in obj.items():
            
#             key_id = id(k)
#             v_id = id(v)
#             for _ in [key_id, v_id]:
#                 if _ not in obj_ids:
#                     obj_ids.add(_)
#                     size+= sys.getsizeof(_)
#         return size

def get_size_of(obj, obj_ids):
    
    
    size = sys.getsizeof(obj)
    if id(obj) in obj_ids:
        return 0, obj_ids
    obj_ids.add(id(obj))
    
    if type(obj) in (str, float, int):
        return size, obj_ids
    
    if type(obj) in (list, tuple):
        for item in obj: 
            res = get_size_of(item, obj_ids)
            size += res[0]
            obj_ids = res[1]
        return size, obj_ids
    
    if type(obj) == (dict):
        for k, v in obj.items():
            for _ in [k,v]:
                res = get_size_of(_, obj_ids)
                size += res[0]
                obj_ids = res[1]
        return size, obj_ids

def get_real_size_of(obj):
    
    obj_ids = set()
    
    return get_size_of(obj, obj_ids)[0]

In [236]:
assert get_real_size_of('aa') + get_real_size_of([]) + 8 == get_real_size_of(['aa'])

### Задача 2 [Многочлены] (0.64 балла)

**Условие:**

Реализовать класс многочлена. Определить операции:

1) *сложения* - **(0.02 балла)**

2) *вычитания* - **(0.02 балла)** 

3) *умножения* - **(0.04 балла)**

3a) *быстрого умножения* (алгоритм Карацубы или быстрое преобразование Фурье) - **(+0.25 балла)**

4) *деления* - **(0.05 балла)** 

5) *возведения в степень* - **(0.02 балла)** 

6) *представления многочлена в человеческом виде* - **(0.02 балла)**

7) *дифференцирования* - **(0.05 балла)**

8) *интегрирования* - **(0.05 балла)**

9) Вызова многочлена как функции (вычисление значения в точке) - **(0.03 балла)**


Операции с числами также должны работать.

In [97]:
0.02+0.02+0.04+0.05+0.02+0.02+0.05+0.05+0.03

0.29999999999999993

In [1]:
#https://stackoverflow.com/questions/5125619/why-doesnt-list-have-safe-get-method-like-dictionary

class safelist(list):
    def get(self, index, default=None):
        try:
            return self.__getitem__(index)
        except IndexError:
            return default

In [419]:
class Polynomial:
    def __init__(self, *coefs):
        """
        Coefs starting from a0 + a1 * x + a2 * x^2 .... 
        """
        self.last_non_zero_coef = self.find_last_non_zero_coef(coefs)
        self.coefs = safelist([float(coef) for coef in coefs[:max(self.last_non_zero_coef+1, 1)]])
        self.length = len(self.coefs)
    
    @staticmethod
    def find_last_non_zero_coef(ls):
        for i in range(len(ls)-1, -1, -1):
            if ls[i] != 0:
                return i 
        return -1 
    
    def linear_operations(self, other, method):
        if type(other) in (int, float):
            other = Polynomial(other)
            
        resulting_coefs = []
        max_len = max(len(self.coefs), len(other.coefs))
        for i in range(max_len):
            result = method(self.coefs.get(i, 0.0), other.coefs.get(i, 0.0))
            resulting_coefs.append(result)
        return resulting_coefs
    
    def __add__(self, other):  
        return Polynomial(*self.linear_operations(other, float.__add__))
        
    def __sub__(self, other):
        return Polynomial(*self.linear_operations(other, float.__sub__))
        
    def __mul__(self, other):
        if type(other) in (int, float):
            other = Polynomial(other)

        resulting_coefs = [0] * (self.length + other.length)        
        if (self.last_non_zero_coef != -1) and (other.last_non_zero_coef != -1):
            for i in range(self.length):
                for j in range(other.length):
                    resulting_coefs[i+j] += self.coefs[i] * other.coefs[j]
                
        return Polynomial(*resulting_coefs)
    
    def __pow__(self, power):
        if power == 1:
            return self 
        if power == 0: 
            return Polynomial(0)
        
        temp_polynomial = self
        for i in range(2, power+1): 
            temp_polynomial *= self
            
        return temp_polynomial
    
    def __call__(self, point):
        if type(point) not in [int, float]:
            raise TypeError('Input should be float or int')
        
        return sum([coef * (point ** i) for i, coef in enumerate(self.coefs)])

    def derivative(self, order = 1):
        if order == 0:
            return self
        if self.length == 1: 
            return Polynomial(0)
        
        resulting_coefs = [0] * (self.length - 1)
        for i in range(1,self.length): 
            resulting_coefs[i-1] = self.coefs[i] * i
            
        return Polynomial(*resulting_coefs).derivative(order -1)
    
    def integral(self):
        
        resulting_coefs = [0] * (self.length + 1)
        
        for i in range(0, self.length):
            resulting_coefs[i+1] = self.coefs[i] / (i + 1)
        
        return Polynomial(*resulting_coefs)
    
    def __truediv__(self, other): 
        if type(other) in (float, int):
            other = Polynomial(other)
            
        temp_poly = self
        result_poly, remainder_poly  = Polynomial(0), Polynomial(0)
        
        num_largest_coef = len(temp_poly.coefs)
        denom_largest_coef = len(other.coefs)
        
        if num_largest_coef == denom_largest_coef == 1:
            return (Polynomial(temp_poly.coefs[0] / other.coefs[0]), remainder_poly)
        
        while num_largest_coef >= denom_largest_coef:
            
            temp_resulting_coefs = [0] * (num_largest_coef - denom_largest_coef + 1) 
            temp_resulting_coefs[-1] = temp_poly.coefs[-1] / other.coefs[-1]
            
            div_poly = Polynomial(*temp_resulting_coefs)
            temp_poly = temp_poly - (div_poly * other)
            result_poly = result_poly + div_poly
            
            num_largest_coef = len(temp_poly.coefs)
            denom_largest_coef = len(other.coefs)
            
        remainder_poly = self - (result_poly * other)
        
        return result_poly, remainder_poly
        
    def __str__(self):
        
        result_string = ""
        for i in range(len(self.coefs)):
            
            leading_spaces = 1 if i > 0 else 0 
            
            if self.coefs[i] >= 0:
                prefix = ' '  * leading_spaces + '+ '* leading_spaces
            else:
                prefix = ' ' * leading_spaces  + '- '
                
            result_string += f"{prefix}{abs(self.coefs[i])}*x^{i}"
            
        return result_string

In [430]:
poly1  = Polynomial(1,1)
poly2 = Polynomial(5,9, 99)

# Check if add and subtr return the same results
poly_sum = poly1 + poly2
assert (poly2).coefs == (poly_sum - poly1).coefs

# Check if integral returns same after derivativee
assert poly1.coefs == poly1.integral().derivative().coefs

# Check if division is correct
div, rem =  poly1 / poly2
assert poly1.coefs ==  (poly2 * div + rem).coefs

# check value at point
assert poly1(0) == poly1.coefs[0]

### Задача 3 [Аналог range] (0.05 балла)

**Условие:**

Реализуйте итератор с поведением, аналогичным range.

In [4]:
class custom_range_iter:
    def __init__(self, *args):
        
        if len(args) == 1:
            self.start, self.stop, self.step = 0, args[0], 1
        elif len(args) == 2: 
            self.start, self.stop, self.step = *args, 1
        else:
            self.start, self.stop, self.step = args
        
        if self.step != 0:
            self.current = self.start - self.step
        else:
            raise ValueError('step should not be zero')
            
    def __iter__(self):
        return self
    
    def __next__(self): 
    
        self.next = self.current + self.step
        if ((self.step > 0 and (self.next < self.stop) and (self.next >= self.start))
           or (self.step <0 and (self.next > self.stop) and (self.next <= self.start))):
            self.current = self.next
            return self.next
        else:
            raise StopIteration
            

In [94]:
from random import randint

N_ATTEMPTS = 1e6

def get_points():
    start, stop, step = randint(-100, 100), randint(-100, 100), randint(-100, 100) // randint(1,5)
    
    if step == 0:
        
        return get_points()
    return start, stop, step 

for _ in range(int(N_ATTEMPTS)):
    start, stop, step = get_points()
    assert list(custom_range_iter(start, stop, step)) == list(range(start, stop, step))

### Задача 4 [Primary Key] (0.05 балла)

**Условие:**

С помощью механизма дескрипторов реализуйте Primary Key - свойства первичного ключа из PostgreSQL.

### Задача 5 [PositiveSmallIntegerField] (0.03 балла)

**Условие:**

С помощью механизма дескрипторов реализуйте тип данных PositiveSmallIntegerField - поле, принимающее значения от 0 до 32767.

In [443]:
class PositiveSmallIntegerField:
    def __init__(self, name):
        self.name = name
        
    def __get__(self, instance, owner):
        return instance.__dict__[self.name]
    
    def __set__(self, instance, value):
        if (isinstance(value, int)) and (0 <= value <= 32767):
            instance.__dict__[self.name] = value
        else:
            raise ValueError('The number should be integer and be in range [0, 32767]')
            
    def __delete__(self, instance):
        instance.__dict__[self.name] = None
            
class UsingPositiveIntegers:

    number = PositiveSmallIntegerField('number')
    
    def __init__(self, number):
        self.number = number

In [444]:
x =UsingPositiveIntegers(10)

In [445]:
values = [-1, 5, 999999, 1.5]
for value in values:
    try:
        test_num = UsingPositiveIntegers(value)
    except Exception as e:
        assert str(e) == 'The number should be integer and be in range [0, 32767]'

### Задача 6 [Timer] (0.02 балла)

**Условие:**

Реализовать контекстный менеджер, который выводит время, проведенное в нём.

In [95]:
import time

class TimeContextManager:
    def __init__(self):
        self.time_entered = None
    def __enter__(self):
        self.time_entered = time.time()
    def __exit__(self, type, value, traceback):
        print(f'Spent  {round(time.time() - self.time_entered, 10)} seconds in the context manager')

In [96]:
with TimeContextManager():
    time.sleep(5)

Spent  5.0050239563 seconds in the context manager
